<a href="https://colab.research.google.com/github/JockWang/colab/blob/master/Molecule%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install catboost

In [0]:
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn

In [0]:
path = '/content/drive/My Drive/Colab Notebooks/molecule/molecule/'
train_target = pd.read_csv(path+'train_answer.csv')
train_feature = pd.read_csv(path+'candidate_train.csv')
test_feature = pd.read_csv(path+'candidate_val.csv')

In [0]:
train = pd.merge(train_feature, train_target, how='left', on='id')

In [6]:
errors = []
columns = [col for col in list(train.columns) if col not in ['id','p1','p2','p3','p4','p5','p6']]
for i in columns:
  cols = train[i]
  mean_ = cols.mean()
  min_ = cols.min()
  max_ = cols.max()
  if mean_ == 0 and min_ == 0 and max_ == 0:
    # print(i)
    errors.append(i)
print(len(errors))

51


In [7]:
columns = [col for col in list(train.columns) if col not in ['id','p1','p2','p3','p4','p5','p6']+errors]
cats = []
for i in columns:
  cols = train[i]
  size = list(set(cols))
  min_ = cols.min()
  max_ = cols.max()
  if len(size) < 10 and int(min_) == min_ and int(max_) == max_:
    cats.append(i)
print('len:',len(cats))

len: 3103


In [0]:
category = errors+cats

#### CatBoost

In [0]:
def CatKfoldResult(X, test, y, cat_features, K=2):
  fold = KFold(n_splits=K,shuffle=True,random_state=2020)
  preds = []
  models = []
  i = 0
  for train_index, valid_index in fold.split(X, y):
      i += 1
      print('第',str(i),'个Fold')
      train_x,valid_x,train_y,valid_y = X.iloc[train_index],X.iloc[valid_index],y.iloc[train_index],y.iloc[valid_index]
      eval_set = cat.Pool(data=valid_x,label=valid_y,cat_features=cat_features)
      model = cat.CatBoostRegressor(iterations=50000, learning_rate=0.1, depth=12, loss_function='MAE', eval_metric='MAE',
                      use_best_model=True, verbose=100, task_type='GPU', devices='0:1')
      model.fit(train_x,train_y,use_best_model=True,eval_set=eval_set,cat_features=cat_features,early_stopping_rounds=100)
      models.append(model)
      preds.append(model.predict(test))
  preds = np.array(preds)
  return preds.mean(axis=0)

In [0]:
test = test_feature.copy()
remove = ['id','p1','p2','p3','p4','p5','p6']
for col in ['p1','p2','p3','p4','p5','p6']:# 
    print(col)
    print(remove)
    features = [col for col in list(train.columns) if col not in remove]
    remove.remove(col)
    X = train[features]
    y = train[col]
    test = test[features]
    model = cat.CatBoostRegressor(iterations=10000, learning_rate=0.1, depth=12, loss_function='MAE', eval_metric='MAE',
                   verbose=100, task_type='GPU', devices='0:1')
    model.fit(X,y,cat_features=category)
    # pred = CatKfoldResult(X=X,y=y,test=test,cat_features=category)
    test[col] = model.predict(test)

p1
['id', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6']
0:	learn: 1.1295312	total: 154ms	remaining: 25m 37s
100:	learn: 0.6174100	total: 12.5s	remaining: 20m 29s
200:	learn: 0.5591425	total: 25.9s	remaining: 21m 3s
300:	learn: 0.5194280	total: 39.4s	remaining: 21m 8s
400:	learn: 0.4862019	total: 52.8s	remaining: 21m 4s
500:	learn: 0.4578389	total: 1m 6s	remaining: 20m 52s
600:	learn: 0.4327600	total: 1m 19s	remaining: 20m 41s
700:	learn: 0.4103252	total: 1m 32s	remaining: 20m 29s
800:	learn: 0.3890388	total: 1m 46s	remaining: 20m 18s
900:	learn: 0.3698601	total: 1m 59s	remaining: 20m 4s
1000:	learn: 0.3523219	total: 2m 12s	remaining: 19m 51s
1100:	learn: 0.3354252	total: 2m 25s	remaining: 19m 38s
1200:	learn: 0.3195591	total: 2m 38s	remaining: 19m 24s
1300:	learn: 0.3057790	total: 2m 52s	remaining: 19m 10s
1400:	learn: 0.2926827	total: 3m 5s	remaining: 18m 56s
1500:	learn: 0.2796854	total: 3m 18s	remaining: 18m 42s
1600:	learn: 0.2678932	total: 3m 31s	remaining: 18m 29s
1700:	learn: 0.2566077	to

#### LGB

In [0]:

def lgbKFold(X,test,p,yarams, K=5):
    fold = KFold(n_splits=K,shuffle=True,random_state=2020)
    preds = []
    valid = 0
    i = 0
    for train_index, valid_index in fold.split(X,y):
        i += 1
        print('第',str(i),'个Fold')
        train_x,valid_x,train_y,valid_y = X.iloc[train_index],X.iloc[valid_index],y.iloc[train_index],y.iloc[valid_index]
        lgb_train = lgb.Dataset(train_x,train_y)
        lgb_valid = lgb.Dataset(valid_x,valid_y)
        model = lgb.train(params,train_set=lgb_train,valid_sets=lgb_valid,early_stopping_rounds=100, verbose_eval=500)
        val_pred = model.predict(valid_x,num_iteration=model.best_iteration)
        temp_valid = mean_absolute_error(valid_y, val_pred)
        print('valid mae_score:%f'%(temp_valid))
        preds.append(model.predict(test,num_iteration=model.best_iteration))
    preds = np.array(preds)
    return preds.mean(axis=0)

In [9]:
test = test_feature.copy()
params = {
    'objective': 'regression_l1',
    'boosting_type': 'gbdt', 
    'num_leaves': 81,
    'max_depth': 12,
    'learning_rate': 0.1,
    'n_estimators': 50000, 
    'subsample': .8,
    'subsample_freq': 1,
    'colsample_bytree': .8,
    'n_threads': -1,
}
remove = ['id','p1','p2','p3','p4','p5','p6']
for col in ['p1','p2','p3','p4','p5','p6']:# 
    print(col)
    print(remove)
    features = [col for col in list(train.columns) if col not in remove]
    remove.remove(col)
    X = train[features]
    y = train[col]
    test = test[features]
    pred = lgbKFold(X=X,y=y,test=test,params=params)
    test[col] = pred

p1
['id', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6']
第 1 个Fold
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's l1: 0.560318
[1000]	valid_0's l1: 0.546698


KeyboardInterrupt: ignored

##### 没有添加EMB的

In [0]:
test = test_feature.copy()
params = {
    'objective': 'regression_l1',
    'boosting_type': 'gbdt', 
    'num_leaves': 81,
    'max_depth': 12,
    'learning_rate': 0.1,
    'n_estimators': 50000, 
    'subsample': .8,
    'subsample_freq': 1,
    'colsample_bytree': .8,
    'n_threads': -1,
}
remove = ['id','p1','p2','p3','p4','p5','p6']
for col in ['p1','p2','p3','p4','p5','p6']:# 
    print(col)
    print(remove)
    features = [col for col in list(new_train.columns) if col not in remove]
    remove.remove(col)
    X = new_train[features]
    y = new_train[col]
    test = test[features]
    pred = lgbKFold(X=X,y=y,test=test,params=params)
    test[col] = pred

p1
['id', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6']
第 1 个Fold


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[500]	valid_0's l1: 0.613593
[1000]	valid_0's l1: 0.595317
[1500]	valid_0's l1: 0.584286
[2000]	valid_0's l1: 0.576609
[2500]	valid_0's l1: 0.570261
[3000]	valid_0's l1: 0.565497
[3500]	valid_0's l1: 0.561559
[4000]	valid_0's l1: 0.558498
[4500]	valid_0's l1: 0.555787
[5000]	valid_0's l1: 0.553216
[5500]	valid_0's l1: 0.551019
[6000]	valid_0's l1: 0.549157
[6500]	valid_0's l1: 0.547364
[7000]	valid_0's l1: 0.545744
[7500]	valid_0's l1: 0.54436
[8000]	valid_0's l1: 0.543152
[8500]	valid_0's l1: 0.542244
[9000]	valid_0's l1: 0.54124
[9500]	valid_0's l1: 0.54031
[10000]	valid_0's l1: 0.539409
[10500]	valid_0's l1: 0.538616
[11000]	valid_0's l1: 0.537959
[11500]	valid_0's l1: 0.537246
[12000]	valid_0's l1: 0.536573
[12500]	valid_0's l1: 0.535951
[13000]	valid_0's l1: 0.535433
[13500]	valid_0's l1: 0.534866
[14000]	valid_0's l1: 0.534307
[14500]	valid_0's l1: 0.53373
[15000]	valid_0's l1: 0.533197
[15500]	valid_0's l1: 0.532555


KeyboardInterrupt: ignored

In [0]:
test = test_feature.copy()
for col in ['p1','p2','p3','p4','p5','p6']:
    print(col)
    remove = ['id','p1','p2','p3','p4','p5','p6']
    features = [col for col in list(train.columns) if col not in remove]
    remove.remove(col)
    X = train[features]
    y = train[col]
    model = lgb.LGBMRegressor(objective='regression_l1',boosting_type='gbdt', num_leaves=81, 
                            max_depth=10,learning_rate=0.01, n_estimators=5000, max_bin=255, 
                            subsample_for_bin=200000, min_split_gain=0.0,
                            min_child_weight=0.001, min_child_samples=20, subsample=.8,
                            subsample_freq=1, colsample_bytree=.9, reg_alpha=0.0,
                            reg_lambda=0.0, random_state=None, n_jobs=-1, silent=True)
    model.fit(X,y)
    inputs = test[features]
    test[col] = model.predict(inputs)

p1
p2
p3
p4
p5
p6


In [0]:
test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3143,3144,3145,3146,3147,3148,3149,3150,3151,3152,3153,3154,3155,3156,3157,3158,3159,3160,3161,3162,3163,3164,3165,3166,3167,3168,3169,3170,3171,3172,3173,3174,3175,3176,p1,p2,p3,p4,p5,p6
0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,112.038511,1.750000,2.500000,2.875000,108.060,0.309464,0.309464,0.263844,-0.263844,0,42,0.473942,0.973945,0.009886,0.226783,0.773217,78.212262,266.664496,344.876756,1.786485,112.092,-1.42970,5,0,60.67,0,3.692847,-0.252881,-0.089484,0.084391,-410.862569,-410.708110
1,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,123.068414,1.444444,2.000000,2.444444,114.083,0.349443,0.222342,0.222342,-0.349443,0,48,0.549482,0.982475,0.011977,0.186395,0.972043,81.159108,423.242443,435.415219,2.029533,123.155,0.59510,2,1,29.10,0,3.881726,-0.236673,0.012790,0.148819,-401.974785,-401.964370
2,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,113.058912,1.625000,2.375000,2.625000,106.064,0.389916,0.110730,0.110730,-0.389916,0,44,0.296281,0.934496,0.005633,0.355973,0.694232,123.254685,240.376174,346.247541,1.829334,113.120,-0.39458,4,2,61.80,1,1.973916,-0.242949,-0.000899,0.120125,-396.327033,-396.033969
3,0,0,0,0,0,0,3,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,118.099380,1.375000,1.875000,2.000000,104.064,0.391019,0.074527,0.074527,-0.391019,0,50,0.587055,0.986423,0.010138,0.164225,0.854223,84.259788,438.281302,513.076246,2.230665,118.176,0.79220,2,1,29.46,3,1.937787,-0.248636,0.076555,0.196097,-387.217731,-387.321642
4,0,0,0,0,0,0,1,0,5,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,123.104799,1.555556,2.222222,2.333333,110.095,0.197722,0.068924,0.068924,-0.197722,0,50,0.210765,0.901111,0.003431,0.433589,0.704075,205.223639,333.248296,473.313759,2.143609,123.199,2.48038,1,0,23.79,2,3.943208,-0.297285,0.034328,0.195008,-367.258320,-367.302616


In [0]:
lgb_submit = test[['p1','p2','p3','p4','p5','p6']]
lgb_submit['id'] = test_feature['id']
lgb_submit = lgb_submit[['id','p1','p2','p3','p4','p5','p6']]
lgb_submit.to_csv('submit_lgb.csv',index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
lgb_submit.head()

,id,p1,p2,p3,p4,p5,p6
0,66b19220,3.692847,-0.252881,-0.089484,0.084391,-410.862569,-410.708110
1,bed4af38,3.881726,-0.236673,0.012790,0.148819,-401.974785,-401.964370
2,96f5658c,1.973916,-0.242949,-0.000899,0.120125,-396.327033,-396.033969
3,d7031639,1.937787,-0.248636,0.076555,0.196097,-387.217731,-387.321642
4,1fdd8f3c,3.943208,-0.297285,0.034328,0.195008,-367.258320,-367.302616


In [0]:
lgb_submit.head()

,id,p1,p2,p3,p4,p5,p6
0,66b19220,3.388132,-0.257876,-0.080408,0.084733,-411.254395,-411.987496
1,bed4af38,3.817437,-0.234795,0.013277,0.149034,-401.987329,-401.962970
2,96f5658c,1.897177,-0.242279,-0.002558,0.120592,-396.195385,-396.456617
3,d7031639,1.982961,-0.249490,0.075534,0.196482,-387.410662,-387.336497
4,1fdd8f3c,3.948038,-0.297728,0.033491,0.193371,-366.747763,-366.771573


In [0]:
test = test_feature.copy()
for col in ['p1','p2','p3','p4','p5','p6']:
    print(col)
    remove = ['id','p1','p2','p3','p4','p5','p6']
    features = [col for col in list(train.columns) if col not in remove]
    remove.remove(col)
    X = train[features]
    y = train[col]
    model = xgb.XGBRegressor(max_depth=12, learning_rate=0.01, n_estimators=1500,
                            silent=True, objective='reg:linear', nthread=-1, 
                            gamma=0, min_child_weight=1, max_delta_step=0, 
                            subsample=.8, colsample_bytree=.8, colsample_bylevel=.8, 
                            reg_alpha=0, reg_lambda=1, scale_pos_weight=1, 
                            base_score=0.5, seed=0, missing=None)
    model.fit(X,y)
    inputs = test[features]
    test[col] = model.predict(inputs)

p1


In [0]:
xgb_submit = test[['id','p1','p2','p3','p4','p5','p6']]
xgb_submit.to_csv('submit_xgb.csv',index=False)

In [0]:
xgb_submit.head()

,id,p1,p2,p3,p4,p5,p6
0,66b19220,0.810120,0.430166,0.446426,0.461680,-38.653721,-38.652992
1,bed4af38,0.803341,0.429844,0.450440,0.466077,-37.988358,-37.987545
2,96f5658c,0.657551,0.431066,0.453279,0.463266,-38.143883,-38.143131
3,d7031639,0.638009,0.429018,0.459019,0.470704,-36.157486,-36.156673
4,1fdd8f3c,0.850918,0.427115,0.454378,0.470889,-34.801277,-34.800201
